In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
past_sales = pd.read_csv('/kaggle/input/rossmann-store-sales/train.csv')
future_sales = pd.read_csv('/kaggle/input/rossmann-store-sales/test.csv')
store = pd.read_csv('/kaggle/input/rossmann-store-sales/store.csv')
past_sales['StateHoliday'] = past_sales['StateHoliday'].replace({0: '0'})
past_sales['Date'] = pd.to_datetime(past_sales['Date'], format='%Y-%m-%d')
past_sales.shape

In [ ]:
past_sales.head()

In [ ]:
past_sales['SchoolHoliday'].unique()

## Summarizing numeric metrics

In [ ]:
num_cols = ['Sales', 'Customers']
past_sales[num_cols].describe()

In [ ]:
past_sales['Sales'].plot.box();

### Summarizing categorical columns

In [ ]:
cat_cols = ['Store', 'DayOfWeek', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday']
past_sales[cat_cols].astype(object).describe()

In [ ]:
past_sales['StateHoliday'].unique()

### Summarizing Date Columns

In [ ]:
past_sales['Date'].describe(datetime_is_numeric=False)

In [ ]:
#past_sales.isna().sum()

In [ ]:
#future_sales.head()
store.head()
#future_sales.isna().sum()

In [ ]:
store_imputed = store.drop(['Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval'], axis=1).copy()
# store_imputed.isna().sum() / store_imputed.shape[0] * 100
avg_compt_dist = store_imputed['CompetitionDistance'].mean()
store_imputed['CompetitionDistance'] = store_imputed['CompetitionDistance'].fillna(avg_compt_dist)

mode_comp_month = store_imputed['CompetitionOpenSinceMonth'].mode().iloc[0]
store_imputed['CompetitionOpenSinceMonth'] = store_imputed['CompetitionOpenSinceMonth'].fillna(
    mode_comp_month)

mode_comp_year = store_imputed['CompetitionOpenSinceYear'].mode().iloc[0]
store_imputed['CompetitionOpenSinceYear'] = store_imputed['CompetitionOpenSinceYear'].fillna(
    mode_comp_year)
#store_imputed.isna().sum() / store_imputed.shape[0] * 100

In [ ]:
past_sales_merged = pd.merge(left=past_sales,
                            right=store_imputed,
                            on=['Store'],
                            how='left')
past_sales_merged.shape

In [ ]:
past_sales_dummies = pd.get_dummies(past_sales_merged, drop_first=True)
past_sales_dummies.shape